# Car Price Estimator

In this notebook we are going to take our first steps in ML with ```scikit-learn```. You are provided with a dataset with cars on sale in CarDekho (a vehicle sale portal from India). Let's see if we can predict the price of a car based on its characteristics!

The dataset was extracted from [this Kaggle](https://www.kaggle.com/nehalbirla/vehicle-dataset-from-cardekho). I strongly recommend visiting this Kaggle and explore the notebooks from other data scientists in the world working with this same data. It's always nice to see the work that other people genuinely share. You can always learn from them, the way they analyze the data, the algorithms they use and the plots they do.

In [1]:
import pandas as pd

df = pd.read_csv("posts.csv")
df

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai i20 Magna,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197 CC,82.85 bhp,113.7Nm@ 4000rpm,5.0
8124,Hyundai Verna CRDi SX,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493 CC,110 bhp,"24@ 1,900-2,750(kgm@ rpm)",5.0
8125,Maruti Swift Dzire ZDi,2009,382000,120000,Diesel,Individual,Manual,First Owner,19.3 kmpl,1248 CC,73.9 bhp,190Nm@ 2000rpm,5.0
8126,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396 CC,70 bhp,140Nm@ 1800-3000rpm,5.0


There are three different CSV files in this dataset, but we will only use the one called *Car details v3.csv*. As we can see, in our dataframe we have a set of characteristics of cars (like the kilometers driven, the year of the car, the mileage, etc) and their selling price.

In [2]:
df.count()

name             8128
year             8128
selling_price    8128
km_driven        8128
fuel             8128
seller_type      8128
transmission     8128
owner            8128
mileage          7907
engine           7907
max_power        7913
torque           7906
seats            7907
dtype: int64

It's always a good start to check for the completeness of our dataset. With the ```count``` function above we can see that there are some missing values, specially in the last columns, but not that many. It's a very complete dataset.

In [3]:
df.dtypes

name              object
year               int64
selling_price      int64
km_driven          int64
fuel              object
seller_type       object
transmission      object
owner             object
mileage           object
engine            object
max_power         object
torque            object
seats            float64
dtype: object

As you already know, the type of each column is inferred in ```read_csv``` method from its values. We can see that in our dataframe most of the columns are *object*'s (strings). Only *year*, *selling_price*, *km_driven* and *seats* are numerical.

Obviously, the number of *seats* is an integer value, but ```pandas``` is treating it as a float. Why is that? Well, this is just because of the missing values in this column. In ```pandas``` missing values are represented by ```np.nan``` which is an special object in ```numpy``` of type *float64*. Thus, when a numerical column contains missing values, the column type must be considered alwaya as *float64*. In ML, there's really no difference between integers and floats, so may all the columns be treated as floats and it would work the same.

### Target and features

When doing supervised ML, the first step is to establish is which one of our columns is going to be the *target* (that means, the column we want to be able to predict). In our case, we are trying to predict the price of the cars so the target will be *selling_price*. This price is in Indian rupees, though, so maybe we can start by converting it into euros (so it would be easier for us to understand it). Today, the exchange rate (says Google) is 1 rupee = 0.012 euros.

In [4]:
df["selling_price"] = df["selling_price"] * 0.012

As the second step, we need to establish which columns we are going to use as *features* (that means, the columns we want to be able to predict with). Something that will always be helpful in order to choose the features is taking a look at the correlation matrix. We are specially interested in the correlations of our *target* against the other columns.

In [5]:
df.corr(numeric_only=True).style.background_gradient("coolwarm", vmin=-1, vmax=1)

,year,selling_price,km_driven,seats
year,1.000000,0.414092,-0.418006,-0.009144
selling_price,0.414092,1.000000,-0.225534,0.041358
km_driven,-0.418006,-0.225534,1.000000,0.227336
seats,-0.009144,0.041358,0.227336,1.000000


We can see here that our target (*selling_price*) has a strong positive correlation with *year* and a not-so-strong-but-still-good negative correlation with *km_driven*. On the other hand, there is no correlation with *seats*. That does makes sense, as we know that the newer the car is, the more it's worth, and the less kilometers it has been driven, the better. Also, we know that the number of seats it's not something relevant to the price.

The correlation matrix gives us a hint of the columns that can be useful for a model. Keep in mind, though, that this matrix is only showing us direct linear correlations, so it's not a universal truth. Some variables can be very useful to a ML model because of a non-linear correlation or because of cross-correlations with other variables.

The best criteria to select the features to our model is common sense and knowledge of the problem you are solving. In this case we **know** that *seats* can't be a relevant feature, and that's why we shouldn't use it in our model, becasue it won't give us anything.

In [6]:
target = "selling_price"
features = ["year", "km_driven"]
X, y = df[features], df[target]

### Train-Test split

The next step is splitting the data in *train* and *test* sets. The *train* data is the data that we are going to use to fit our model, whilst the *test* data is the data that we will use test our model and see how good it is.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (6502, 2)
X_test shape: (1626, 2)
y_train shape: (6502,)
y_test shape: (1626,)


The easiest way to perform the train-test split is through the ```train_test_split``` method in ```sklearn```. In the cell above we took 20% of our data to test (and 80% to train). The test size you should take is something for you to decide. The bigger the train set, the better your model will learn, but the bigger the test set is, the more solid your metrics will be. It depends on the volume of data you have, but typically taking 10% or 20% for testing is good enough.

This train-test split method from ```sklearn``` is dividing **randomly** your data. Imagine your data is sorted by selling price and you split in the order your data is: your model will be trained with cars with low selling price and you will be testing with cars with a high price!! That's why it is important to randomly split your data. The fourth argument ```random_state``` fixes the random seed to use so you can reexecute the line and get the same result.

### Linear Model

Let's create our first ML model with the simplest algorithm there is in the world: Linear Regression.

In [8]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression()

Just instantiate the model and fit with the training data. With the ```fit``` method, the ```linear_model``` adjusted its parameters to have the minimal squared error against the target. In a linear model, you know there are only a few parameters, the coefficient for each feature and the intercept. You can check them out if you want:

In [9]:
linear_model.coef_, linear_model.intercept_

(array([ 9.27105050e+02, -1.56799094e-02]), np.float64(-1858159.5572970135))

So this is just telling us that our model will estimate the price of a car with this formula:

$$ \hat{selling\_price} = -1,858,199.5572 + 927.10505 * year - 0.0156799094 * km\_driven $$

Now ```linear_model``` is a ready to predict and we can try it out with our test data.

In [10]:
linear_y_pred = linear_model.predict(X_test)

results_df = X_test.copy()
results_df["y_real"] = y_test
results_df["y_pred"] = linear_y_pred.astype(int)
results_df["err"] = results_df["y_real"] - results_df["y_pred"]
results_df["%_err"] = results_df["err"] / results_df["y_real"] * 100
results_df

,year,km_driven,y_real,y_pred,err,%_err
1392,2005,80000,3000.0,-568,3568.0,118.933333
7778,2017,45000,15384.0,11105,4279.0,27.814613
3727,2019,60000,8400.0,12724,-4324.0,-51.476190
6630,2020,15000,6000.0,14357,-8357.0,-139.283333
103,2016,100000,3240.0,9316,-6076.0,-187.530864
...,...,...,...,...,...,...
3697,2010,120000,2604.0,3440,-836.0,-32.104455
3001,2012,110000,4200.0,5451,-1251.0,-29.785714
94,2009,55500,2100.0,3524,-1424.0,-67.809524
4535,2015,15000,9600.0,9721,-121.0,-1.260417


The table above show us the results on the test data. You can see here with just a glance that the results are horrible. There's even cars whose predicted selling price is negative!! It's quite a crappy model, but don't be harsh. It's just a linear regression after all, with only *year* and *km_driven* as features.

To be able to numerically tell how good or bad a model is you have to use *metrics*. There are many of them, and they depend on the type of problem you are solving. For instance, in a binary classification problem you would use ROC AUC, while in regression problems (like the one we are doing here) you would typically go for RMSE. All these metrics are available in ```sklearn.metrics```.

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

print(f"RMSE: {mean_squared_error(y_test, linear_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, linear_y_pred)}")
print(f"R^2: {r2_score(y_test, linear_y_pred)}")

RMSE: 7698.548290468255
MAPE: 0.8988796385823179
R^2: 0.16323632984102243


The RMSE tells us about the average error in our predictions: we are failing by 7698€ on average. The MAPE tells us about the average percentage error, since 7698€ is not the same in a car priced at 200k€ than in a 5k€ car: in our case we are failing by 90% on average. Finally the R^2 score is the Pearson's coefficient of predictions against reality: 1 would be a perfect model, while 0 is worse than a monkey predicting the price through what the tarot cards tell him.

### Gradient Boosting Machines

Choosing the right model for the problem you have to solve is important. Although linear models are simple and great and they have been a very important milestone in scientific history, today there are much more sophisticated algorithms. Today, the state of the art for tabular data (both for regression and classification) are *Gradient Boosting Machines*. In very simple words, they are like a chain of decision trees where each one estimates the error from the previous one.

There are many implementations of GBM, and ```sklearn``` has its own. Although for a professional case I would recommend using specific libraries like [LightGBM](https://lightgbm.readthedocs.io/en/v3.3.2/), [XGBoost](https://xgboost.readthedocs.io/en/stable/) or [CatBoost](https://catboost.ai/), for the purposes of this course the ```sklearn```'s implementation ir more than enough.

In [12]:
from sklearn.ensemble import GradientBoostingRegressor

gbm_model = GradientBoostingRegressor()
gbm_model.fit(X_train, y_train)
gbm_y_pred = gbm_model.predict(X_test)

results_df = X_test.copy()
results_df["y_real"] = y_test
results_df["y_pred"] = gbm_y_pred.astype(int)
results_df["err"] = results_df["y_real"] - results_df["y_pred"]
results_df["%_err"] = results_df["err"] / results_df["y_real"] * 100
results_df

,year,km_driven,y_real,y_pred,err,%_err
1392,2005,80000,3000.0,1601,1399.0,46.633333
7778,2017,45000,15384.0,21436,-6052.0,-39.339574
3727,2019,60000,8400.0,9176,-776.0,-9.238095
6630,2020,15000,6000.0,7161,-1161.0,-19.350000
103,2016,100000,3240.0,8468,-5228.0,-161.358025
...,...,...,...,...,...,...
3697,2010,120000,2604.0,3314,-710.0,-27.265745
3001,2012,110000,4200.0,4250,-50.0,-1.190476
94,2009,55500,2100.0,2586,-486.0,-23.142857
4535,2015,15000,9600.0,5592,4008.0,41.750000


You can see that the results are better now (at least there are no negative predictions). Let's compare the metrics: 

In [13]:
print(f"RMSE: {mean_squared_error(y_test, gbm_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, gbm_y_pred)}")
print(f"R^2: {r2_score(y_test, gbm_y_pred)}")

RMSE: 5892.946433522567
MAPE: 0.564421830935943
R^2: 0.5097132841642201


According to what we saw before, the metrics are all remarkably better.

### Improving the model

The GBM model we have just created is still far from perfect. But, would you be able to better predict the price of a car with just *year* and *km_driven*?

Our model is lacking from features. As a general rule, the more (and better) features the better the model is going to perform. You can go for the most sophisticated algorithm in the world, but if you don't have good features you won't be able to predict nothing. That's the key role of the data scientist: find and create good features for your model. This process is commonly referred as *feature engineering*.  

And the truth is that our dataset contains many other information.

In [14]:
df

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,5400.0,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,4440.0,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,1896.0,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,2700.0,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,1560.0,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai i20 Magna,2013,3840.0,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197 CC,82.85 bhp,113.7Nm@ 4000rpm,5.0
8124,Hyundai Verna CRDi SX,2007,1620.0,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493 CC,110 bhp,"24@ 1,900-2,750(kgm@ rpm)",5.0
8125,Maruti Swift Dzire ZDi,2009,4584.0,120000,Diesel,Individual,Manual,First Owner,19.3 kmpl,1248 CC,73.9 bhp,190Nm@ 2000rpm,5.0
8126,Tata Indigo CR4,2013,3480.0,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396 CC,70 bhp,140Nm@ 1800-3000rpm,5.0


The thing is that we cannot pass to the algorithm any non-numerical information, and most of our columns are strings. Can we do something about it? 

Well, first of all, we have columns with strings in the form "X units". There is a number that we must be able to extract somehow. Check it out:

In [15]:
df["mileage"].str.split().str[0].astype(float)

0       23.40
1       21.14
2       17.70
3       23.00
4       16.10
        ...  
8123    18.50
8124    16.80
8125    19.30
8126    23.57
8127    23.57
Name: mileage, Length: 8128, dtype: float64

Same applies to *engine* and *max_power*. Let's create columns for these as floats:

In [16]:
df["mileage_asfloat"] = df["mileage"].str.split().str[0].astype(float)
df["engine_asfloat"] = df["engine"].str.split().str[0].astype(float)
df["max_power_asfloat"] = df["max_power"].str.split().str[0].astype(float)

ValueError: could not convert string to float: 'bhp'

Here we can see Python complaining about converting 'bhp' to a float. If you dig around a little bit, you'll see that the problem is that there is a row in our dataset with *max_power* equal to ' bhp'.

In [ ]:
df.loc[df["max_power"] == " bhp"]

That's something weird. But these things happen, you'll see that data is not perfect and many times (if not always) you need to first make some cleaning. That's just life, guys. Get used to it.

In this case I'm just going to put a NaN in this field.

In [ ]:
import numpy as np

df.loc[df["max_power"] == " bhp", "max_power"] = np.nan

Now this should go ok:

In [ ]:
df["max_power_asfloat"] = df["max_power"].str.split().str[0].astype(float)

Now our dataframe has 3 more numerical columns that we can use in our model.

In [ ]:
df[["mileage_asfloat", "engine_asfloat", "max_power_asfloat"]]

But there are still some other columns that we can convert into numerical. I'm talking about *fuel*, *seller_type*, *transmission* and *owner*. These are all what we call *categorical columns*. There are a few possible values and all the individuals belong to one of these. These categorical columns can also be converted into numbers assigning a number to each category. See how can we do it in pandas:

In [ ]:
df["fuel"]

In [ ]:
df["fuel"].astype("category").cat.codes

It is true, though, that using this type of encoding can be a little bit misleading. These numbers are assigned "randomly" and there is no relation between the order of the numbers and the characteristics of the categories. In the example above 'CNG' was given the 0, 'Diesel' is 1, 'LPG' is 2 and 'Petrol' is 3: is really 'CNG' closer to 'Diesel' than to 'Petrol'?

Some purists prefer encoding with tools like [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html?highlight=onehot#sklearn.preprocessing.OneHotEncoder) but I don't want to get into that right now. If you use a tree-model like GBM, the values of the codes is not very very important (I don't say it is not at all). So, for today, we will do this thing to all categorical columns.

In [ ]:
df["fuel_encoded"] = df["fuel"].astype("category").cat.codes
df["seller_type_encoded"] = df["seller_type"].astype("category").cat.codes
df["transmission_encoded"] = df["transmission"].astype("category").cat.codes

The column *owner*, however, is worth to be treated specially. Although it is a categorical column, the values of these categories do have a clear order:

In [ ]:
df["owner"].unique()

In a case like this, you maybe want to put the codes manually like this:

In [ ]:
df["owner_encoded"] = df["owner"].map({
    'Test Drive Car': 0,
    'First Owner': 1,
    'Second Owner': 2,
    'Third Owner': 3,
    'Fourth & Above Owner': 4,
})

At this point, we now have many numerical columns in our dataset. Let's see the correlation matrix.

In [ ]:
df.corr(numeric_only=True).style.background_gradient("coolwarm", vmin=-1, vmax=1)

We can see that some of the new columns present a good correlation with our target, like *max_power_asfloat* or *transmission_encoded*. Let's try now a new GBM with all these features.

In [ ]:
features = [
    "year",
    "km_driven",
    "seats",
    "engine_asfloat",
    "max_power_asfloat",
    "mileage_asfloat",
    "fuel_encoded",
    "seller_type_encoded",
    "transmission_encoded",
    "owner_encoded",
]

Before that, however, we must first do something about missings values. Models in ```sklearn``` cannot handle NaN values, and thus we have to get rid of them. This problem did not came up before because  ```year``` and ```km_driven``` were complete columns, but that's not the case for many of the new columns we've created. 

There are mainly two options about NaN's. First option is just drop any rows containing missing values in the features selected.

In [ ]:
df.dropna(subset=features)

The second option is to fill the NaN values with something. You can fill NaNs with a fixed value like 0 or an atypical value like -99999 so the model can learn these are something special, or you can fill them with the mean value of the corresponding column, so the model will assume "normality" for the missing value.

In [ ]:
df.fillna(-9999)  # This fills all NaNs with -9999
df.fillna(df.mean(numeric_only=True))  # This fills NaNs with the mean in each column

In this case, since the number of rows with missing values is not that much, we will use ```dropna```.

In [ ]:
_df = df.dropna(subset=features)
X = _df[features]
y = _df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

Now we can train and test our GBM.

In [ ]:
gbm_model = GradientBoostingRegressor()
gbm_model.fit(X_train, y_train)
gbm_y_pred = gbm_model.predict(X_test)

print(f"RMSE: {mean_squared_error(y_test, gbm_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, gbm_y_pred)}")
print(f"R^2: {r2_score(y_test, gbm_y_pred)}")

And you can see that the results now are way way better. Now this is a pretty good model!!

### Tuning hyperparameters

Hyperparameters are the config parameters of an algorithm. As opposed to ML model internal parameters, these parameters cannot be adjusted to your data through the ```fit``` method, and it's something that the data scientists must decide prior to train.

For instance, in a GBM model, one hyperparameter is the number of trees or the depth of these trees. Each model has its own hyperparameters that you must know in order to properly adjust. Check the documentation of the algorithm you are using to know more about them.

These hyperparameters are typically set at model instantiation. Let's run the same model as before but changing some of them.

In [ ]:
gbm_model = GradientBoostingRegressor(n_estimators=200, max_depth=6, random_state=15)
gbm_model.fit(X_train, y_train)
gbm_y_pred = gbm_model.predict(X_test)

print(f"RMSE: {mean_squared_error(y_test, gbm_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, gbm_y_pred)}")
print(f"R^2: {r2_score(y_test, gbm_y_pred)}")

The results have improved! In this case we have increased the number of trees (default is 100) and making them more deep (default depth is 3). These two hyperparameters will increase the complexity of the model, increasing also the training time. You should know, though, that more complexity will not always mean more better performance. When a model gets too complex (specially decisions trees) overfitting is a issue.

In [ ]:
_gbm_model = GradientBoostingRegressor(n_estimators=300, max_depth=10, random_state=15)
_gbm_model.fit(X_train, y_train)
_gbm_y_pred = _gbm_model.predict(X_test)

print(f"RMSE: {mean_squared_error(y_test, _gbm_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, _gbm_y_pred)}")
print(f"R^2: {r2_score(y_test, _gbm_y_pred)}")

There are no universal rules to decide the proper hyperparameters for a model. Only when you are a real expert (much more than I am) you develop the clinical eye to know what hyperparameters will be the best.

The vast majority of people just go with the default or perform something called *grid search*. *Grid search* is just a fancy name for *trial and error*: you choose an array of different values of each hyperparameter, create a grid with all different combinations, try them all and keep the model with best metrics. I'm not a big fan of grid search because when you do that your are using the test data to adjust your model, which is something *philosophically incorrect*.

### Plots about model performance

In addition to metrics, plots are also a great tool to see how your model performs. Also as with metrics, the kind of plots that you may want to draw depends on the kind of problem you are solving. For binary classifications, the ROC curve and Precision-Recall plots cannot be missing. For regressions problems like the one we are dealing with today, the essential plot is *predictions* vs *reality*.

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
ax.set_title("Model performance")
ax.set_ylabel("Pred value")
ax.set_xlabel("Real value")
ax.set_yscale("log")
ax.set_xscale("log")
ax.scatter(y_test, gbm_y_pred, s=1)
plot_range = [y_test.min(), y_test.max()]
ax.plot(plot_range, plot_range, c="red")
plt.show()

Of course, the more the points of this graph fit the line, the better the model will be. Sometimes with this plot you can also see *where* your model fails the most. For instance, in our model we can see that that the points are further from the line at low values of the real target.

Another plot that I like to draw is the target and the prediction distributions to see if the model is capturing the global behaviour of the target.  

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
ax.set_title("Target distributions")
ax.set_ylabel("Pred value")
ax.set_xlabel("Real value")
ax.set_yscale("log")
ax.hist(y_test, bins=50, color="blue", alpha=0.5, label="Real")
ax.hist(gbm_y_pred, bins=50, color="orange", alpha=0.5, label="Prediction")
ax.legend()
plt.show()

Finally, the error's distribution.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
ax.set_title("Model error distribution")
ax.set_ylabel("Number of cases")
ax.set_xlabel("Error")
ax.set_yscale("log")
ax.hist(y_test - gbm_y_pred, bins=50)
plt.show()

### Analysing the errors

Your model will never be perfect, that is a reality. Metrics will tell you how good is your model but always in a global way. In order to improve your model, it is important that you try to understand where your model is failing and why.

In [ ]:
preds_df = df.loc[y_test.index].copy()
preds_df["y_real"] = preds_df[target]  # I'm just adding this column to have both y_real and y_pred at the right of my df
preds_df["y_pred"] = gbm_y_pred
preds_df["err"] = preds_df["y_real"] - preds_df["y_pred"]
preds_df["abs_err"] = preds_df["err"].abs()
preds_df["%_err"] = preds_df["err"] / preds_df["y_real"] * 100
preds_df["abs_%_err"] = preds_df["%_err"].abs()
preds_df

In [ ]:
preds_df.sort_values("abs_err", ascending=False).head(5)

Now at this point is where the *business knowledge* comes in. *Business knowledge* is consultant jargon (I'm a consultant), we refer this way to the part of the problem that our client knows more. I don't know nothing about cars, I already told you, but I could be working as a data scientist consultant for CarDekho (why not?) and at this point I would send them this table for them to tell me if this data is consistent or not.

Is it logical that Skoda Kodiaq from 2018 (2 years old, this dataset was extracted in 2020) and with this characteristics was priced at 33,600€? Or does it make more sense that its price was around 13k? To me, ignorant, that sounds expensive. If its correct that the price should be 33k, then why? Is there any extra information that we are missing? Did this car belong to Michael Jackson or something like that?

Same questions apply to all other cars with excessive errors. With the proper answer, we may find some other information that can be relevant, add it to the model, and get even better metrics.

Another interesting thing when analysing errors is dividing into groups. For instance:

In [ ]:
preds_df.groupby("seller_type")[["y_real", "y_pred", "abs_err", "abs_%_err"]].agg(["mean", "std", "count"])

Again, I don't know about this topic, and I have literally no idea what a *Trustmark Dealer* is, but it is evident that we are nailing these cars.

Or maybe we can go for something more interesting like the car brand (assuming the car brand is the first word in the car name).

In [ ]:
preds_df["brand"] = preds_df["name"].str.split().str[0]
error_by_brand = preds_df.groupby("brand")[["y_real", "y_pred", "abs_err", "abs_%_err"]].agg(["mean", "std", "count"])
error_by_brand.sort_values(("abs_%_err", "mean"))

In [ ]:
error_by_brand.loc[
    error_by_brand[("abs_%_err", "count")] > 5
][("abs_%_err", "mean")].sort_values(ascending=False).plot(kind="bar")

We do fail a lot in Skoda, that's weird, right?

### Feature importances

Another part that cannot be missing in a ML model is feature importance analysis. In order to understand the model that you have just created, you have to know which where the most relevant features.

Most of ML algorithms provide some *way* to tell about the importance of the features. Sometimes it's something super clear like in linear models, where the importance of course will be directly related to the coefficients of each feature; sometimes it can be a little more difficult to tell, like in GBM's where, since the result comes from decisions trees, it's not clear what's the specific contribution of each feature. 

In [ ]:
gbm_model.feature_importances_

In the GBM implementation from ```sklearn``` we have this attribute that gives us what is called the *Gini importance*. This is a measure of the importance of each feature based on the number of splits (across all trees) that include this feature. As said, that is just a *type of measure* for importance, although is not always clear what *importance* means in this context. As always, a plot is nicer to see the data.

In [ ]:
importances = pd.Series(gbm_model.feature_importances_, index=features)
importances.sort_values(ascending=True).plot(kind="barh")

So what we see in here is that *max_power* is clearly the most important feature in our data to predict the price, followed by *year*. It's remarkable how low the importance of *km_driven* is (in my ignorant opinion). The features at the bottom of this chart show no importance and probably the model would work exactly the same if we do not consider them.

Another common measure of importance is *permutation importance*. This is a technique that is appliable to any model (no matter how opaque it is) where the data is tabular. This is calculated for each feature by evaluating the difference in model performance when the feature is randomly shuffled. If a feature is really important, the model performance should decrease a lot when this variable is given random, while, on the other hand, when a feature is really not important, the model should perform the same when the feature is given random.

You can import ```permutation_importance``` from ```sklearn``` too. This calculation of features importances can take a little bit longer, but I honestly trust this values a little bit more than Gini importance.

In [ ]:
from sklearn.inspection import permutation_importance

imp = permutation_importance(gbm_model, X_test, y_test, n_repeats=10, random_state=0)

In [ ]:
imp.importances_mean, imp.importances_std

In [ ]:
importances = pd.Series(imp.importances_mean, index=features)
importances.sort_values(ascending=True).plot(kind="barh")

The results are similar, that's expectable, of course. But you can see here that *km_driven* is higher than *engine* and *fuel* has some impact, while *seats* proved to be non-important at all.

Finally, there's this other really new and really cool tool called [shap](https://shap.readthedocs.io/en/latest/index.html). It's a library that aimes to give model explainability with a game theoretic approach and they have some really good results. Remember to install this library before continuing.

In [ ]:
import shap

shap.initjs()  # Most of shap plots are interactive and require some JS that is loaded with this line

In [ ]:
explainer = shap.TreeExplainer(gbm_model)

In [ ]:
%%time
# This operation takes a while
shap_values = explainer.shap_values(X_test)

One of the cool thing about shap is that it give us the importance (or contribution) that each feature had in each prediction. Individually, not globally. These are the values that we stored in the ```shap_values``` variables. Now, you can ask for how the contributions where for each single prediction. Let's the first one, for instance.

In [ ]:
preds_df.head()

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0,:], X_test.iloc[0,:])

In this plot we can see that the feature that was the most important in this prediction was the year of the car (2011, almost ten years old). That pushed down the prediction almost 2k€. Secondly, having 73.9 for *max_power* (relatively low) also pushed down a lot. Also, being from fourth owner or above (*owner_encoded* = 4) and having more than 90k kilometers driven pushed down. Being diesel (*fuel_encoded* = 1) pushed it up a little bit, but not so much.

There are many other interesting visualizations in shap.

In [ ]:
shap.summary_plot(shap_values, X_test)

Summary plot shows the importance of all features in all predictions (each point is a prediction). Surely, shap values are giving us another measure of importance. We can plot an importance plot similar to the ones we did before like this:

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

Finally, a super interesting plot from shap are *dependency plots*.

In [ ]:
shap.dependence_plot("year", shap_values, X_test)

This is plotting the shap value (contribution) of *year* feature in each one of our predictions againts the actual value of the *year* feature. In addition, this plot uses color to represent another feature. This other feature is automatically chosen by ```shap``` to be the one with most interaction.

First thing to notice here is that the contribution of this feature has an ascending trend only starting in approx 2010; in cars older than this, the contribution is roughly constant (negative). That is telling us that the price will change if the car is from 2015 or 2018, but it is going to be pretty much the same if it is from 2000 or from 1995.

Secondly, we can see a special interaction between *year* and *max_power*. It seems that, in recent years (>~2017) the higher the *max_power*, the higher the contribution of this (recent) year. On the other hand, in older years (between ~2005 and ~2015) it seems that the higher the *max_power*, the lower (when we say the lower we mean the higher in negative) the *year* contribution is. The insight is clear: the higher the *max_power* of the car, the more important the *year* becomes.

### Conclusions

I think this is a very complete example of how to do ML the good way. Remember the main steps:

1. Choose the target.
2. Analyze the data you have to select the best features.
3. Do the train-test split.
4. Choose a proper algorithm and train your model.
5. Test your model and see the metrics.
6. Analyze model errors, features importances and, if you want more, go back to point 2 again.

I'll leave you with the final metrics of our model, just to brag a little bit more.

In [ ]:
print(f"RMSE: {mean_squared_error(y_test, gbm_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, gbm_y_pred)}")
print(f"R^2: {r2_score(y_test, gbm_y_pred)}")

JFYI, the most voted notebook in this Kaggle scored 0.94, we did 0.977 😉.